# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages


In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
import pprint

with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

pprint.pprint(twits['data'][:15])

[{'message_body': '$FITB great buy at 26.00...ill wait',
  'sentiment': 2,
  'timestamp': '2018-07-01T00:00:09Z'},
 {'message_body': '@StockTwits $MSFT',
  'sentiment': 1,
  'timestamp': '2018-07-01T00:00:42Z'},
 {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a '
                  'rating of Hold setting target price at USD 350.00. Our own '
                  'verdict is Buy  http://www.stocktargetadvisor.com/toprating',
  'sentiment': 2,
  'timestamp': '2018-07-01T00:01:24Z'},
 {'message_body': '$AMD I heard there’s a guy who knows someone who thinks '
                  'somebody knows something - on StockTwits.',
  'sentiment': 1,
  'timestamp': '2018-07-01T00:01:47Z'},
 {'message_body': '$AMD reveal yourself!',
  'sentiment': 0,
  'timestamp': '2018-07-01T00:02:13Z'},
 {'message_body': '$AAPL Why the drop? I warren Buffet taking out his '
                  'position?',
  'sentiment': 1,
  'timestamp': '2018-07-01T00:03:10Z'},
 {'message_body': '$BA bears have

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.
#### DONE

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [5]:
pprint.pprint(messages[:15])
pprint.pprint(sentiments[:15])

['$FITB great buy at 26.00...ill wait',
 '@StockTwits $MSFT',
 '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold '
 'setting target price at USD 350.00. Our own verdict is Buy  '
 'http://www.stocktargetadvisor.com/toprating',
 '$AMD I heard there’s a guy who knows someone who thinks somebody knows '
 'something - on StockTwits.',
 '$AMD reveal yourself!',
 '$AAPL Why the drop? I warren Buffet taking out his position?',
 '$BA bears have 1 reason on 06-29 to pay more attention '
 'https://dividendbot.com?s=BA',
 '$BAC ok good we&#39;re not dropping in price over the weekend, lol',
 '$AMAT - Daily Chart, we need to get back to above 50.',
 '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... '
 'if BO, then bingo... what is the odds?',
 '$SBUX STRONG BUY!',
 '$SNPS short ratio is 2.17 at 2018-06-15 and short % to float is 1.42% '
 'http://sunshineavenue.com/stock/SNPS/ via @sunshineave',
 '$NFLX price squeezing,perfect place for an option stra

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

#### DONE

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/toilethill/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    text = message.lower()

    url_pattern = re.compile(
        r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)',
        re.IGNORECASE
    )
    text = re.sub(url_pattern, ' ', text)
    
    symb_pattern = re.compile(r'[@#$][\w_-]+', re.IGNORECASE)
    text = re.sub(symb_pattern, ' ', text)
    
    not_alph_pattern = re.compile(r'[\W\d]')
    text = re.sub(not_alph_pattern, ' ', text)
    
    single_char = re.compile(r'\s[\w]\s')
    text = re.sub(single_char, ' ', text)
    
    tknizr = TweetTokenizer()
    tokenized_words = tknizr.tokenize(text)
    tokens = [w for w in tokenized_words]
    
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w) for w in tokens if len(w) > 1]
    
    return tokens

# 13
# @DEEPAKM2013 @Nytunes Start of new Q on Monday. Expect strong buy volume across key companies of various sectors. $AMZN $AAPL
# 14
# [BREAKOUT Strategy] Current Portfolio : $ZBRA,$WEB,$TIF,$SRE,$SPPI,$OTEX,$OMF,$NVGS,$NRCIB,$MSG,$KIRK,$GHDX,$FBNK,$ESND,$DRI,$DKS,$CVTI,$CV

msg = messages[14]
preprocess(msg)

['breakout', 'strategy', 'current', 'portfolio']

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

#### DONE

In [7]:
############################
### Loading and processing all the Twits crashes the notebook kernal
############################
import json
from tqdm import tqdm 
from pathlib import Path

datastore = Path("../../data/project_6_stocktwits/datastore")

def twit_batcher(batches, twits):
    """
    Preprocessing the large amount of Twits all at once kept collapsing the kernel.
    - This function:
        - Breaks the process up into batches of 10,000 twits.
        - Every twit in a batch is tokenized with the preprocess() function.
        - Each batch of tokenized twits is saved as json file in the datastore directory.
    
    Parameters
    ----------
        batches : The number of batches to process.
        twits: The messages or twits to be processed.
        
    """
    tail = 0
    
    batched = tqdm(range(batches), desc=f'Saving twits batch...')
    for batch in batched:
        head = tail + 10000
        file_name = f'{tail}:{head}.json'
        ds = datastore / json / file_name
        tokenized = []
        for twit in twits[tail:head]:
            tokenized.append(preprocess(twit))
        with ds.open('w+') as f:
            json.dump(tokenized, f)
            f.close()
        tail = head

In [8]:
total_msgs = len(messages)
batches = round(total_msgs/10000)

# twit_batcher(batches, messages)
print(f'Total Msgs Preprocessed: {total_msgs}, In {batches} batches.')

Total Msgs Preprocessed: 1548010, In 155 batches.


### Bag of Words 


Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.



#### DONE

In [10]:
import pickle

In [13]:
from collections import Counter

"""
Create a vocabulary by using Bag of words
"""
 
def bag_of_words(words):
    bow = Counter()
    
    for word in words:
        bow[word] += 1
    return bow

def datastore_retriever():
    json_datastore = datastore / 'json'
    stored_files = [f for f in json_datastore.iterdir()]
    tokenized = []
    words = []
    
    for file_path in stored_files:
        with open(file_path, 'rb') as f: 
            tokenized += json.load(f)
            f.close()
            
    for msg in tokenized:
        words += msg

    bow = bag_of_words(words)

    return bow, tokenized
    
with open(datastore / 'pickled' / 'data.pickle', 'wb+') as f:
    bow, tokenized = datastore_retriever()
    data = {'bow': bow, 'tokenized': tokenized}
    pickle.dump(data, f)
    f.close()

In [11]:
with open(datastore / 'pickled' / 'data.pickle', 'rb') as f:
    data = pickle.load(f)
    tokenized = data['tokenized']
    bow = data['bow']
    f.close()

In [12]:
print(len(tokenized))
print(len(bow))

1548010
105283


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.


#### DONE 

In [17]:
"""
Set the following variables:
    freqs = Dictionart that contains the Frequency of words appearing in messages. \
            The key is the token and the value is the frequency of that word in the corpus.
    low_cutoff = Float that is the frequency cutoff. Drop words with a frequency that is lower or equal \
            to this number.
    high_cutoff = Integer that is the cut off for most common words. Drop words that are the `high_cutoff` \
            most common words.
    K_most_common = The k most common words in the corpus. Use `high_cutoff` as the k.
"""
# TODO Implement 
total_count = len(bow)
freqs = {word: count/total_count for word, count in bow.items()}

low_cutoff = 1e-5
high_cutoff = 10
K_most_common = bow.most_common(high_cutoff)

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(len(tokenized))
print(K_most_common)
len(filtered_words) 
print(filtered_words[:50])

1548010
[('the', 398828), ('to', 379789), ('amp', 295510), ('is', 284902), ('for', 273569), ('on', 241672), ('of', 211367), ('and', 208527), ('in', 205367), ('this', 203576)]
['yeeees', 'had', 'been', 'head', 'scratcher', 'for', 'day', 'waiting', 'look', 'like', 'suitable', 'er', 'trade', 'candidate', 'current', 'implied', 'move', 'of', 'omg', 'about', 'to', 'overtake', 'in', 'market', 'cap', 'mean', 'top', 'is', 'this', 'it', 'green', 'can', 'someone', 'livestream', 'today', 'earning', 'report', 'short', 'getting', 'their', 'buttholes', 'plastered', 'estimate', 'distribution', 'here', 'what', 'estimize', 'analyst', 'are', 'expecting']


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.


#### DONE

In [18]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement
# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {}

for word_id, word in enumerate(filtered_words, 1):
    vocab[word] = word_id
    id2vocab[word_id] = word
    
# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in tknzd_msg if word in filtered_words] for tknzd_msg in tokenized]

In [ ]:
# print(len(filtered))
# with open(datastore / 'pickled' / 'filtered.pickle', 'wb+') as f:
#     pickle.dump(filtered, f)
#     f.close()

In [ ]:
# with open(datastore / 'pickled' / 'filtered.pickle', 'rb') as f:
#     filtered = pickle.load(f)
#     f.close()

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [19]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

# print(n_neutral)
# print(N_examples)
# print(keep_prob)
# print(len(filtered))

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [20]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.20024493718357397

Finally let's convert our tokens into integer ids which we can pass to the network.

In [21]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

print(len(token_ids))

1023936


## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.


#### DONE 

In [22]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


In [32]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.

        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """

        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout

        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers,
                            dropout=dropout, batch_first=False)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_size, output_size)
        self.logsm = nn.LogSoftmax(dim=1)

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state

        Parameters
        ----------
            batch_size : The size of batches.

        Returns
        -------
            hidden_state

        """

        # TODO Implement

        # Create two new tensors with sizes lstm_layers x batch_size x lstm_size,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())

        return hidden

    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.

        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        # TODO Implement

        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden_state)

        # stack up lstm outputs
        lstm_out = lstm_out[-1,:,:]
        # lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)

        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # Softmax function
        logps = self.logsm(out)

        batch_size = nn_input.size(1)
        
        return logps, hidden_state

### View Model

In [33]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-2.0000, -1.9232, -1.4399, -1.5827, -1.2867],
        [-2.0058, -1.9190, -1.4263, -1.5828, -1.2979],
        [-2.0089, -1.9272, -1.4158, -1.6004, -1.2882],
        [-1.9806, -1.9537, -1.3976, -1.5805, -1.3197]],
       grad_fn=<LogSoftmaxBackward>)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [34]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

#### DONE 

In [35]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_frac = 0.7

split_idx = int(len(token_ids)*split_frac)
train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]

In [36]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

#### TODO

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.25)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

cpu


TextClassifier(
  (embedding): Embedding(48578, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.25)
  (dropout): Dropout(p=0.25)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (logsm): LogSoftmax()
)

In [41]:
from tqdm import tqdm
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 5
batch_size = 512
learning_rate = 1e-3
clip=5
print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):

    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size):
        steps += 1
        hidden = model.init_hidden(labels.shape[0])

        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)

        # ------------------------------

        # TODO Implement: Train Model
        # zero accumulated gradients
        model.zero_grad()

        # get the output from the model
        output, hidden = model(text_batch, hidden)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        # loss stats
        if steps % print_every == 0:
            # Get validation loss
            model.eval()
            val_losses = []

            dl_iterable = tqdm(
                dataloader(
                    valid_features, valid_labels, batch_size=batch_size),
                desc=f'Epoch{epoch}')
            for inputs, labels in dl_iterable:
                inputs, labels = inputs.to(device), labels.to(device)

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = model.init_hidden(labels.shape[0])
                for vh in val_h:
                    vh.to(device)

                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)
                val_losses.append(val_loss.item())

            # TODO Implement: Print metrics
            model.train()
            print(
                f'Epoch:{epoch}/{epochs} | Step:{steps} | Loss:{loss.item()} | Val Loss:{val_losses}'
            )


Epoch0: 0it [00:00, ?it/s]
Epoch0: 1it [00:01,  1.55s/it]
Epoch0: 2it [00:03,  1.54s/it]
Epoch0: 3it [00:04,  1.53s/it]
Epoch0: 4it [00:06,  1.50s/it]
Epoch0: 5it [00:07,  1.62s/it]
Epoch0: 6it [00:09,  1.74s/it]
Epoch0: 7it [00:12,  1.88s/it]
Epoch0: 8it [00:13,  1.82s/it]
Epoch0: 9it [00:15,  1.72s/it]
Epoch0: 10it [00:16,  1.71s/it]
Epoch0: 11it [00:19,  1.95s/it]
Epoch0: 12it [00:21,  2.01s/it]
Epoch0: 13it [00:23,  1.92s/it]
Epoch0: 14it [00:25,  1.92s/it]
Epoch0: 15it [00:27,  1.96s/it]
Epoch0: 16it [00:29,  1.97s/it]
Epoch0: 17it [00:31,  2.11s/it]
Epoch0: 18it [00:33,  2.13s/it]
Epoch0: 19it [00:35,  1.93s/it]
Epoch0: 20it [00:36,  1.79s/it]
Epoch0: 21it [00:38,  1.70s/it]
Epoch0: 22it [00:40,  1.81s/it]
Epoch0: 23it [00:42,  1.81s/it]
Epoch0: 24it [00:43,  1.69s/it]
Epoch0: 25it [00:45,  1.62s/it]
Epoch0: 26it [00:46,  1.58s/it]
Epoch0: 27it [00:48,  1.54s/it]
Epoch0: 28it [00:49,  1.52s/it]
Epoch0: 29it [00:51,  1.55s/it]
Epoch0: 30it [00:52,  1.63s/it]
Epoch0: 31it [00:54, 

Epoch0: 230it [05:48,  1.46s/it]
Epoch0: 231it [05:49,  1.45s/it]
Epoch0: 232it [05:51,  1.43s/it]
Epoch0: 233it [05:52,  1.42s/it]
Epoch0: 234it [05:54,  1.43s/it]
Epoch0: 235it [05:55,  1.41s/it]
Epoch0: 236it [05:56,  1.42s/it]
Epoch0: 237it [05:58,  1.42s/it]
Epoch0: 238it [05:59,  1.42s/it]
Epoch0: 239it [06:01,  1.43s/it]
Epoch0: 240it [06:02,  1.42s/it]
Epoch0: 241it [06:04,  1.41s/it]
Epoch0: 242it [06:05,  1.41s/it]
Epoch0: 243it [06:06,  1.41s/it]
Epoch0: 244it [06:08,  1.41s/it]
Epoch0: 245it [06:10,  1.54s/it]
Epoch0: 246it [06:11,  1.56s/it]
Epoch0: 247it [06:13,  1.65s/it]
Epoch0: 248it [06:15,  1.68s/it]
Epoch0: 249it [06:16,  1.67s/it]
Epoch0: 250it [06:18,  1.70s/it]
Epoch0: 251it [06:20,  1.71s/it]
Epoch0: 252it [06:22,  1.75s/it]
Epoch0: 253it [06:23,  1.70s/it]
Epoch0: 254it [06:25,  1.61s/it]
Epoch0: 255it [06:26,  1.54s/it]
Epoch0: 256it [06:28,  1.49s/it]
Epoch0: 257it [06:29,  1.47s/it]
Epoch0: 258it [06:30,  1.47s/it]
Epoch0: 259it [06:32,  1.45s/it]
Epoch0: 26

Epoch0: 457it [11:58,  1.64s/it]
Epoch0: 458it [11:59,  1.63s/it]
Epoch0: 459it [12:01,  1.70s/it]
Epoch0: 460it [12:03,  1.62s/it]
Epoch0: 461it [12:04,  1.66s/it]
Epoch0: 462it [12:06,  1.62s/it]
Epoch0: 463it [12:07,  1.55s/it]
Epoch0: 464it [12:09,  1.51s/it]
Epoch0: 465it [12:10,  1.48s/it]
Epoch0: 466it [12:12,  1.46s/it]
Epoch0: 467it [12:13,  1.44s/it]
Epoch0: 468it [12:14,  1.43s/it]
Epoch0: 469it [12:16,  1.43s/it]
Epoch0: 470it [12:17,  1.41s/it]
Epoch0: 471it [12:19,  1.43s/it]
Epoch0: 472it [12:20,  1.42s/it]
Epoch0: 473it [12:21,  1.41s/it]
Epoch0: 474it [12:23,  1.40s/it]
Epoch0: 475it [12:25,  1.72s/it]
Epoch0: 476it [12:27,  1.63s/it]
Epoch0: 477it [12:28,  1.58s/it]
Epoch0: 478it [12:30,  1.52s/it]
Epoch0: 479it [12:31,  1.48s/it]
Epoch0: 480it [12:32,  1.48s/it]
Epoch0: 481it [12:34,  1.60s/it]
Epoch0: 482it [12:36,  1.74s/it]
Epoch0: 483it [12:38,  1.78s/it]
Epoch0: 484it [12:41,  1.96s/it]
Epoch0: 485it [12:43,  2.05s/it]
Epoch0: 486it [12:45,  1.98s/it]
Epoch0: 48

Epoch:0/5 | Step:100 | Loss:1.5904422998428345 | Val Loss:[1.596571683883667, 1.6006900072097778, 1.5821270942687988, 1.575398564338684, 1.6219381093978882, 1.559645652770996, 1.599774718284607, 1.546059489250183, 1.5856761932373047, 1.578657627105713, 1.593709111213684, 1.5612999200820923, 1.5561953783035278, 1.567997932434082, 1.5304763317108154, 1.5804202556610107, 1.6075135469436646, 1.6195214986801147, 1.6394063234329224, 1.6071664094924927, 1.5810134410858154, 1.5802761316299438, 1.6292409896850586, 1.6298199892044067, 1.618003249168396, 1.5859076976776123, 1.574668049812317, 1.5521728992462158, 1.5657720565795898, 1.5816484689712524, 1.5607744455337524, 1.5611661672592163, 1.55447256565094, 1.560917615890503, 1.5934633016586304, 1.6114534139633179, 1.6045563220977783, 1.5870389938354492, 1.6062930822372437, 1.5451747179031372, 1.5756155252456665, 1.6221622228622437, 1.6115338802337646, 1.6183881759643555, 1.5784622430801392, 1.5662542581558228, 1.6060640811920166, 1.659342527389


Epoch0: 0it [00:00, ?it/s]
Epoch0: 1it [00:01,  1.42s/it]
Epoch0: 2it [00:02,  1.42s/it]
Epoch0: 3it [00:04,  1.41s/it]
Epoch0: 4it [00:05,  1.41s/it]
Epoch0: 5it [00:07,  1.46s/it]
Epoch0: 6it [00:08,  1.45s/it]
Epoch0: 7it [00:10,  1.44s/it]
Epoch0: 8it [00:11,  1.44s/it]
Epoch0: 9it [00:12,  1.42s/it]
Epoch0: 10it [00:14,  1.42s/it]
Epoch0: 11it [00:15,  1.42s/it]
Epoch0: 12it [00:17,  1.47s/it]
Epoch0: 13it [00:18,  1.45s/it]
Epoch0: 14it [00:20,  1.44s/it]
Epoch0: 15it [00:21,  1.43s/it]
Epoch0: 16it [00:22,  1.42s/it]
Epoch0: 17it [00:24,  1.43s/it]
Epoch0: 18it [00:25,  1.43s/it]
Epoch0: 19it [00:27,  1.43s/it]
Epoch0: 20it [00:28,  1.42s/it]
Epoch0: 21it [00:30,  1.42s/it]
Epoch0: 22it [00:31,  1.41s/it]
Epoch0: 23it [00:32,  1.40s/it]
Epoch0: 24it [00:34,  1.40s/it]
Epoch0: 25it [00:35,  1.44s/it]
Epoch0: 26it [00:37,  1.45s/it]
Epoch0: 27it [00:38,  1.43s/it]
Epoch0: 28it [00:40,  1.43s/it]
Epoch0: 29it [00:41,  1.44s/it]
Epoch0: 30it [00:42,  1.43s/it]
Epoch0: 31it [00:44, 

...

Epoch:1/5 | Step:400 | Loss:1.5405480861663818 | Val Loss:[1.5732192993164062, 1.5685027837753296, 1.5658395290374756, 1.56650710105896, 1.5871121883392334, 1.5418654680252075, 1.5734126567840576, 1.5218678712844849, 1.5793437957763672, 1.5437304973602295, 1.563539743423462, 1.5298235416412354, 1.524522066116333, 1.549458622932434, 1.4848196506500244, 1.5460509061813354, 1.5628089904785156, 1.606341004371643, 1.617229700088501, 1.5953789949417114, 1.583264708518982, 1.5727893114089966, 1.6176261901855469, 1.5996029376983643, 1.5917764902114868, 1.556199073791504, 1.54084312915802, 1.5264732837677002, 1.5311795473098755, 1.5600450038909912, 1.5332036018371582, 1.5351699590682983, 1.5153510570526123, 1.546539306640625, 1.5798944234848022, 1.5681787729263306, 1.5801268815994263, 1.562006950378418, 1.5807231664657593, 1.526766061782837, 1.5645579099655151, 1.6053078174591064, 1.5884935855865479, 1.5923482179641724, 1.555063009262085, 1.5207455158233643, 1.5770270824432373, 1.63463747501373


Epoch1: 0it [00:00, ?it/s]
Epoch1: 1it [00:01,  1.56s/it]
Epoch1: 2it [00:03,  1.68s/it]
Epoch1: 3it [00:05,  1.63s/it]
Epoch1: 4it [00:06,  1.65s/it]
Epoch1: 5it [00:08,  1.79s/it]
Epoch1: 6it [00:10,  1.83s/it]
Epoch1: 7it [00:12,  1.81s/it]
Epoch1: 8it [00:14,  1.94s/it]
Epoch1: 9it [00:16,  1.81s/it]
Epoch1: 10it [00:17,  1.70s/it]
Epoch1: 11it [00:19,  1.68s/it]
Epoch1: 12it [00:21,  1.97s/it]
Epoch1: 13it [00:23,  1.87s/it]
Epoch1: 14it [00:25,  1.82s/it]
Epoch1: 15it [00:27,  1.80s/it]
Epoch1: 16it [00:29,  1.89s/it]
Epoch1: 17it [00:30,  1.81s/it]
Epoch1: 18it [00:32,  1.72s/it]
Epoch1: 19it [00:34,  1.71s/it]
Epoch1: 20it [00:35,  1.64s/it]
Epoch1: 21it [00:37,  1.62s/it]
Epoch1: 22it [00:38,  1.55s/it]
Epoch1: 23it [00:40,  1.55s/it]
Epoch1: 24it [00:41,  1.57s/it]
Epoch1: 25it [00:43,  1.58s/it]
Epoch1: 26it [00:45,  1.70s/it]
Epoch1: 27it [00:47,  1.75s/it]
Epoch1: 28it [00:50,  2.11s/it]
Epoch1: 29it [00:52,  2.26s/it]
Epoch1: 30it [00:54,  2.26s/it]
Epoch1: 31it [00:57, 

Epoch1: 230it [07:23,  1.74s/it]
Epoch1: 231it [07:25,  1.74s/it]
Epoch1: 232it [07:26,  1.69s/it]
Epoch1: 233it [07:28,  1.61s/it]
Epoch1: 234it [07:29,  1.58s/it]
Epoch1: 235it [07:31,  1.59s/it]
Epoch1: 236it [07:32,  1.67s/it]
Epoch1: 237it [07:34,  1.71s/it]
Epoch1: 238it [07:36,  1.65s/it]
Epoch1: 239it [07:37,  1.59s/it]
Epoch1: 240it [07:39,  1.69s/it]
Epoch1: 241it [07:41,  1.71s/it]
Epoch1: 242it [07:43,  1.68s/it]
Epoch1: 243it [07:44,  1.71s/it]
Epoch1: 244it [07:46,  1.66s/it]
Epoch1: 245it [07:48,  1.65s/it]
Epoch1: 246it [07:49,  1.62s/it]
Epoch1: 247it [07:51,  1.70s/it]
Epoch1: 248it [07:53,  1.69s/it]
Epoch1: 249it [07:54,  1.73s/it]
Epoch1: 250it [07:56,  1.76s/it]
Epoch1: 251it [07:58,  1.77s/it]
Epoch1: 252it [08:00,  1.77s/it]
Epoch1: 253it [08:02,  1.79s/it]
Epoch1: 254it [08:03,  1.70s/it]
Epoch1: 255it [08:05,  1.69s/it]
Epoch1: 256it [08:07,  1.82s/it]
Epoch1: 257it [08:09,  1.76s/it]
Epoch1: 258it [08:10,  1.70s/it]
Epoch1: 259it [08:12,  1.71s/it]
Epoch1: 26

Epoch1: 457it [15:02,  1.86s/it]
Epoch1: 458it [15:04,  1.76s/it]
Epoch1: 459it [15:05,  1.79s/it]
Epoch1: 460it [15:07,  1.84s/it]
Epoch1: 461it [15:09,  1.79s/it]
Epoch1: 462it [15:11,  1.72s/it]
Epoch1: 463it [15:13,  1.90s/it]
Epoch1: 464it [15:15,  1.83s/it]
Epoch1: 465it [15:16,  1.79s/it]
Epoch1: 466it [15:18,  1.85s/it]
Epoch1: 467it [15:20,  1.95s/it]
Epoch1: 468it [15:23,  2.10s/it]
Epoch1: 469it [15:26,  2.28s/it]
Epoch1: 470it [15:28,  2.38s/it]
Epoch1: 471it [15:31,  2.38s/it]
Epoch1: 472it [15:32,  2.16s/it]
Epoch1: 473it [15:34,  2.02s/it]
Epoch1: 474it [15:36,  2.00s/it]
Epoch1: 475it [15:38,  1.92s/it]
Epoch1: 476it [15:39,  1.78s/it]
Epoch1: 477it [15:41,  1.86s/it]
Epoch1: 478it [15:43,  1.78s/it]
Epoch1: 479it [15:45,  1.84s/it]
Epoch1: 480it [15:47,  1.92s/it]
Epoch1: 481it [15:49,  1.92s/it]
Epoch1: 482it [15:51,  2.03s/it]
Epoch1: 483it [15:53,  1.95s/it]
Epoch1: 484it [15:55,  1.89s/it]
Epoch1: 485it [15:56,  1.85s/it]
Epoch1: 486it [15:58,  1.88s/it]
Epoch1: 48

KeyboardInterrupt: 

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

#### TODO

In [ ]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]
    # Convert words to ids
    tokens = [vocab[word] if word is not None else "sell" for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1, 1)))
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [ ]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

In [ ]:
import numpy as np
tokens = preprocess(text)
tokens = np.asarray(tokens)
print(tokens.view())


### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

#### TODO

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [ ]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [ ]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

Using the `prediction` function, let's apply it to a stream of twits.

In [ ]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                print(twit)
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [ ]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

for i in range(0, 5):
    print(next(score_stream))

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.